#### `Spherical` wave

This is another solution to Helmholtz equation

$$\boxed{U(\mathbf{r})=\frac{e^{i k |\textbf{r}|}}{4\pi|\mathbf{r}|}}$$

* All $\textbf{r}$ having a `constant` distance from origin form a wavefront (i.e., a sphere)
* The time-dependent expression, with wavenumber $k$ and angular frequency $\omega$, is

$$U(\mathbf{r},t)=\frac{e^{i k |\textbf{r}|}}{4\pi|\mathbf{r}|}e^{-i\omega t}$$

* `Wave number` $k=\frac{2\pi}{\lambda}$, and `phase velocity` $c=\frac{\omega}{k}$, here, there's no need to specify $k$ as a vector as the propagation direction is either $\mathbf{r}$ (for source) or $-\mathbf{r}$ (for sink)

In [4]:
import sympy as smp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc, Rectangle, Ellipse, Circle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
np.random.seed(42)
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})
plt.style.use('dark_background')

In [5]:
class SphericalWave:
    def __init__(self, k, omega, dt, duration, direction='source'):
        if k == 0:
            raise ValueError('k must be non-zero')

        if direction not in ['source', 'sink']:
            raise ValueError('direction must be either "source" or "sink"')

        self.k = k
        self.omega = omega
        self.dt = dt
        self.duration = duration
        self.direction = direction # source or sink

        self.x = np.linspace(-3*np.pi, 3*np.pi, 300)
        self.y = np.linspace(-3*np.pi, 3*np.pi, 300)
        self.xx, self.yy = np.meshgrid(self.x, self.y)

        self.wavelength = 2*np.pi/self.k

    def plot_wave(self, time):
        """handles the actual wave plotting at a given time"""
        # compute distance r at each grid
        self.abs_r = np.sqrt(self.xx**2 + self.yy**2)
        # compute wave value at each grid
        if self.direction == 'source':
            self.wave = (1/(4*np.pi*self.abs_r) * np.exp(1j * (self.k * self.abs_r - self.omega * time))).real
        else:
            self.wave = (1/(4*np.pi*self.abs_r) * np.exp(1j * (-self.k * self.abs_r - self.omega * time))).real

        self.ax.clear()
        # clip levels to improve contrast
        self.level_clip = 1
        self.levels = self.level_clip*np.linspace(-1/(4*np.pi*np.min(self.abs_r)), 1/(4*np.pi*np.min(self.abs_r)), 300)
        self.c = self.ax.contourf(self.xx, self.yy, self.wave, levels=self.levels, cmap='seismic')
        if not hasattr(self, 'colorbar'):  # Only add colorbar once
            self.colorbar = self.fig.colorbar(self.c, ax=self.ax, label='Wave Amplitude')

        self.ax.set_aspect('equal', adjustable='box')
        self.ax.set_xlabel('x')
        self.ax.set_ylabel('y')
        self.ax.set_title(f'$\Delta t$: {time:.2f} s, $\omega$: {self.omega} rad/s, $k$: {self.k} \n temporal phase shift: {self.omega*time/(2*np.pi):.2f} cycle(s)')

        self.ax.quiver(0, 0, np.sin(np.pi/4)*self.wavelength, np.cos(np.pi/4)*self.wavelength, angles='xy', scale_units='xy', scale=1, color='r', width=0.01, alpha=0.8, label='$\lambda$')
        self.ax.legend()
        # self.ax.grid(True)
        self.fig.tight_layout()

    def main(self, animation=False):
        """main method to initialize plot and decides whether to animate or just show static plot."""
        self.animation = animation
        self.fig, self.ax = plt.subplots(figsize=(6, 5)) # this only needs setup once
        self.plot_wave(0) # plot at time 0 if not otherwise specified

        if self.animation:
            return self.animate() # this calls self.animate() and returns HTML object
        else:
            plt.show()

    def update(self, frame):
        """used for updating the plot in each frame during the animation"""
        self.plot_wave(frame)

    def animate(self):
        """set up and return the animation"""
        self.num_frames = int(self.duration / self.dt)
        self.ani = FuncAnimation(self.fig, self.update, frames=np.linspace(0, self.duration, self.num_frames), blit=False)
        plt.close(self.fig)
        return HTML(self.ani.to_jshtml())

In [6]:
spherical_wave = SphericalWave(k=3, omega=1, dt=0.25, duration=4*np.pi, direction='source')
spherical_wave.main(animation=True)